# Análise de Doadores de Sangue

Este notebook realiza o processamento, análise exploratória e classificação dos doadores para prever a **disponibilidade para doação**.

In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report, confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

# Configurações gerais
pd.set_option('display.max_columns', None)
sns.set(style='whitegrid')


In [ ]:

# Substitua pelo caminho correto do seu arquivo CSV
df = pd.read_csv("./dataset/blood_donor_dataset.csv")  #
df.head()


In [ ]:

df.info()
df.describe()


In [ ]:

# Distribuição da variável alvo
plt.figure(figsize=(6,4))
sns.countplot(data=df, x='availability')
plt.title('Distribuição da Disponibilidade para Doação')
plt.show()

# Relação entre número de doações e disponibilidade
plt.figure(figsize=(6,4))
sns.boxplot(data=df, x='availability', y='number_of_donation')
plt.title('Número de Doações vs Disponibilidade')
plt.show()


In [ ]:

# Seleção de features
features = ["months_since_first_donation", "number_of_donation", "pints_donated", "blood_group"]
target = "availability"

X = df[features]
y = df[target]

# Codificação da variável categórica
encoder = OneHotEncoder(drop='first', sparse_output=False)
blood_group_encoded = encoder.fit_transform(X[['blood_group']])
blood_group_df = pd.DataFrame(blood_group_encoded, columns=encoder.get_feature_names_out(['blood_group']))
X = X.drop(columns=['blood_group']).reset_index(drop=True)
X = pd.concat([X, blood_group_df], axis=1)

# Codificação do alvo
label_enc = LabelEncoder()
y = label_enc.fit_transform(y)

# Divisão treino/teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)


In [ ]:

models = {
    "KNN": KNeighborsClassifier(n_neighbors=5),
    "Decision Tree": DecisionTreeClassifier(random_state=42),
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42)
}

results = []

for name, model in models.items():
    print(f"\n=== {name} ===")
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    acc = accuracy_score(y_test, y_pred)
    roc = roc_auc_score(y_test, y_pred)
    cv = cross_val_score(model, X, y, cv=5).mean()

    print(f"Accuracy: {acc:.4f}")
    print(f"ROC AUC: {roc:.4f}")
    print(f"Cross-Validation (5-fold): {cv:.4f}")
    print("Confusion Matrix:")
    print(confusion_matrix(y_test, y_pred))
    print("\nClassification Report:")
    print(classification_report(y_test, y_pred, target_names=label_enc.classes_))

    results.append({"Model": name, "Accuracy": acc, "ROC AUC": roc, "CV Mean": cv})

results_df = pd.DataFrame(results)
results_df
